In [3]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset
import pyaudio
import wave
from pytube import YouTube, Playlist
import gc

torch.cuda.empty_cache()
gc.collect()
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print('Device used is: ',device)
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32
print('torch_dtype is: ',torch_dtype)

model_id = "openai/whisper-large-v3"
model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to('cuda')
processor = AutoProcessor.from_pretrained(model_id)
pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
)

def record_chunk(p, stream, chunk_file):
    frames = []
    for i in range(0,int(rate/chunk*2)):
        data = stream.read(chunk)
        frames.append(data)
    wf = wave.open(chunk_file,'wb')
    wf.setnchannels(channel) 
    wf.setsampwidth(p.get_sample_size(pyaudio.paInt16))
    wf.setframerate(rate)
    wf.writeframes(b''.join(frames))
    wf.close()


inp = input('Enter 1 for microphone and 2 for youtube audio')
if(inp==1):
    final_text = ''
    p = pyaudio.PyAudio()
    chunk = 1024
    channel = 1
    rate = 50000
    stream = p.open(format=pyaudio.paInt16, 
                    channels=channel, 
                    rate=rate, 
                    input=True, 
                    frames_per_buffer=chunk)

    try:
        while True:
            chunk_file = 'hello.mp3'
            record_chunk(p,stream,chunk_file)
            result = pipe("hello.mp3",generate_kwargs={"task": "translate"})
            print(result)
            final_text+=result['text']
    except Exception as e:
        print("Done",e)
    finally:
        stream.stop_stream()
        stream.close()
        p.terminate()
        print(final_text)
else:
    yt = YouTube("https://www.youtube.com/watch?v=4KI9BBW_aP8")
    
    # yt = YouTube("https://www.youtube.com/watch?v=dd1kN_myNDs")
    stream = yt.streams.filter(only_audio=True)[0]
    stream.download(filename="audio_english2.mp3")
    result = pipe("audio_english2.mp3",generate_kwargs={"task": "translate"})
    print(result)

Device used is:  cuda:0
torch_dtype is:  torch.float16


Some weights of WhisperForConditionalGeneration were not initialized from the model checkpoint at openai/whisper-large-v3 and are newly initialized: ['proj_out.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ValueError: Wrong index found for <|0.02|>: should be None but found 50366.